# Домашнее задание 3. Парсинг, Git и тестирование на Python

**Цели задания:**

* Освоить базовые подходы к web-scraping с библиотеками `requests` и `BeautisulSoup`: навигация по страницам, извлечение HTML-элементов, парсинг.
* Научиться автоматизировать задачи с использованием библиотеки `schedule`.
* Попрактиковаться в использовании Git и оформлении проектов на GitHub.
* Написать и запустить простые юнит-тесты с использованием `pytest`.


В этом домашнем задании вы разработаете систему для автоматического сбора данных о книгах с сайта [Books to Scrape](http://books.toscrape.com). Нужно реализовать функции для парсинга всех страниц сайта, извлечения информации о книгах, автоматического ежедневного запуска задачи и сохранения результата.

Важной частью задания станет оформление проекта: вы создадите репозиторий на GitHub, оформите `README.md`, добавите артефакты (код, данные, отчеты) и напишете базовые тесты на `pytest`.



In [1]:
#! pip install -q schedule pytest

In [12]:
# Библиотеки, которые могут вам понадобиться
# При необходимости расширяйте список
import re
import time
from datetime import datetime, timedelta

import requests
import schedule
from bs4 import BeautifulSoup

from typing import List, Dict, Any

## Задание 1. Сбор данных об одной книге (20 баллов)

В этом задании мы начнем подготовку скрипта для парсинга информации о книгах со страниц каталога сайта [Books to Scrape](https://books.toscrape.com/).

Для начала реализуйте функцию `get_book_data`, которая будет получать данные о книге с одной страницы (например, с [этой](http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html)). Соберите всю информацию, включая название, цену, рейтинг, количество в наличии, описание и дополнительные характеристики из таблицы Product Information. Результат достаточно вернуть в виде словаря.

**Не забывайте про соблюдение PEP-8** — помимо качественно написанного кода важно также документировать функции по стандарту:
* кратко описать, что она делает и для чего нужна;
* какие входные аргументы принимает, какого они типа и что означают по смыслу;
* аналогично описать возвращаемые значения.

*P. S. Состав, количество аргументов функции и тип возвращаемого значения можете менять как вам удобно. То, что написано ниже в шаблоне — лишь пример.*

In [13]:
 def get_book_data(book_url: str) -> dict:
    """
    Получает данные о книге с веб-страницы:
    название, цену, рейтинг, наличие, описание и дополнительные характеристики из таблицы.
    
    Args:
        book_url (str): URL страницы книги
        
    Returns:
        dict: Словарь с данными о книге. Структура:
            - 'name' (str): Название книги
            - 'price' (str): Цена книги
            - 'rating' (str): Рейтинг книги
            - 'quantity_in_stock' (str): Информация о наличии
            - 'description' (str): Описание книги
            - 'extra_feature' (dict): Дополнительные характеристики
    Raises:
        requests.RequestException: При ошибках сетевого запроса.    
    """

    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    try:
        response = requests.get(book_url, timeout=10)
        response.encoding = 'utf-8'
        response.raise_for_status()
        all_feat = {}
        soup = BeautifulSoup(response.text, 'html.parser')

        # Название книги
        name_el = soup.find('h1')
        all_feat['name'] = name_el.get_text().strip() if name_el else None

        # Цена
        price_el = soup.find('p', {'class': "price_color"})
        all_feat['price'] = price_el.get_text() if price_el else None

        # Рейтинг
        rating_el = soup.find('p', {'class': "star-rating"})
        all_feat['rating'] = rating_el.get('class')[-1] if rating_el else None

        # Количество в наличии
        qt = re.compile(r'\d+')
        availability_el = soup.find('p', {'class': "instock availability"})
        
        if availability_el:
            availability = availability_el.get_text()
            all_feat['quantity_in_stock'] = re.search(qt, availability).group()
        else:
            all_feat['quantity_in_stock'] = None

        # Описание
        description_header = soup.find('div', {'id':'product_description'})
        
        if description_header:
            description_el = description_header.find_next_sibling('p')
            all_feat['description'] = description_el.get_text() if description_el else None
        else:
            all_feat['description'] = None

        # Дополнительные характеристики из таблицы
        features = soup.find('table', {'class':'table'}).find_all('tr')
        extra_feat = {}
        
        if features:
            for feature in features:
                th_el = feature.find('th')
                td_el = feature.find('td')
                extra_feat[th_el.get_text().strip().lower()] = td_el.get_text().strip() if th_el and td_el else None   
                
        all_feat['extra_feature'] = extra_feat
        
        return all_feat
        
    except requests.RequestException as e:
        print(f'Ошибка при сетевом запросе: {e}')
        return {}
        
    except Exception as e:
        print(f'Ошибка при парсинге: {e}')
        return {}
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [14]:
# Используйте для самопроверки
book_url = 'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'
get_book_data(book_url)

{'name': 'A Light in the Attic',
 'price': '£51.77',
 'rating': 'Three',
 'quantity_in_stock': '22',
 'description': "It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love that Silverstein. Need proof of his genius? RockabyeRockabye baby, in the treetopDon't you know a treetopIs no safe place to rock?And

## Задание 2. Сбор данных обо всех книгах (20 баллов)

Создайте функцию `scrape_books`, которая будет проходиться по всем страницам из каталога (вида `http://books.toscrape.com/catalogue/page-{N}.html`) и осуществлять парсинг всех страниц в цикле, используя ранее написанную `get_book_data`.

Добавьте аргумент-флаг, который будет отвечать за сохранение результата в файл: если он будет равен `True`, то информация сохранится в ту же папку в файл `books_data.txt`; иначе шаг сохранения будет пропущен.

**Также не забывайте про соблюдение PEP-8**

In [15]:
def scrape_books(is_save: bool = False) -> List[Dict[str, Any]]:
    """
    Осуществлять парсинг всех страниц 

    Args: 
        is_save (bool): Аргумент-флаг, отвечающий за сохранение данных книг в файл 

    Returns:
        List[Dict[str, Any]]: Список словарей с данными о книгах
    """

    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    num_page = 1
    ref_book = []
    all_books = []
    
    while True:
        base_url = f'https://books.toscrape.com/catalogue/page-{num_page}.html'
        
        try:
            response = requests.get(base_url, timeout=10)
            
            if response.status_code == 404:
                break

            response.raise_for_status()
            response.encoding = 'utf-8'
            soup = BeautifulSoup(response.text, 'html.parser')  
            books_page = soup.find_all('article', {'class':'product_pod'})
            
            # Формируем список ссылок на книги
            if books_page:
                for book in books_page:
                    link = book.find('a')
                    if link and link.get('href'):
                        ref_book.append(link['href'])
                    else:
                        continue
            else:
                continue
                
            num_page += 1
            
        except requests.RequestException as e:
            print(f'Ошибка при запросе страницы {num_page}: {e}')
            break

    # Получаем данные для каждой книги
    for ref in ref_book:
        book_url = 'http://books.toscrape.com/catalogue/' + ref 
        book_data = get_book_data(book_url)
        if book_data:
            all_books.append(book_data)
            
    #Сохранение результата в файл
    if is_save:
        with open('../artifacts/books_data.txt', 'w', encoding='utf-8') as file:
            for book in all_books:
                file.write(f'{book}\n')
    
    return all_books
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [16]:
# Проверка работоспособности функции
res = scrape_books(is_save=True) # Допишите ваши аргументы
print(type(res), len(res)) # и проверки

<class 'list'> 1000


## Задание 3. Настройка регулярной выгрузки (10 баллов)

Настройте автоматический запуск функции сбора данных каждый день в 19:00.
Для автоматизации используйте библиотеку `schedule`. Функция должна запускаться в указанное время и сохранять обновленные данные в текстовый файл.



Бесконечный цикл должен обеспечивать постоянное ожидание времени для запуска задачи и выполнять ее по расписанию. Однако чтобы не перегружать систему, стоит подумать о том, чтобы выполнять проверку нужного времени не постоянно, а раз в какой-то промежуток. В этом вам может помочь `time.sleep(...)`.

Проверьте работоспособность кода локально на любом времени чч:мм.



In [17]:
# НАЧАЛО ВАШЕГО РЕШЕНИЯ
schedule.clear()

def scrape_save():
    """Запускает сбор данных и сохраняет их в файл."""
    
    print("Запуск функции сбора данных", flush=True)
    scrape_books(is_save=True)
    print('Сбор и сохранение данных завершено', flush=True)

# Автоматический запуск функции в 19:00
schedule.every().day.at("19:00").do(scrape_save)

# Проверка кода в ближайшее время
test_time = (datetime.now() + timedelta(minutes=1)).strftime("%H:%M")
schedule.every().day.at(test_time).do(scrape_save)

while True:
    schedule.run_pending()
    time.sleep(1)
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

Запуск функции сбора данных
Сбор и сохранение данных завершено


KeyboardInterrupt: 

## Задание 4. Написание автотестов (15 баллов)

Создайте минимум три автотеста для ключевых функций парсинга — например, `get_book_data` и `scrape_books`. Идеи проверок (можете использовать свои):

* данные о книге возвращаются в виде словаря с нужными ключами;
* список ссылок или количество собранных книг соответствует ожиданиям;
* значения отдельных полей (например, `title`) корректны.

Оформите тесты в отдельном скрипте `tests/test_scraper.py`, используйте библиотеку `pytest`. Убедитесь, что тесты проходят успешно при запуске из терминала командой `pytest`.

Также выведите результат их выполнения в ячейке ниже.

**Не забывайте про соблюдение PEP-8**


In [19]:
# Ячейка для демонстрации работоспособности
# Сам код напишите в отдельном скрипте
!py -m pytest ../tests/test_scraper.py -v

============================= test session starts =============================
platform win32 -- Python 3.11.2, pytest-8.4.2, pluggy-1.6.0 -- C:\Users\User\AppData\Local\Programs\Python\Python311\python.exe
cachedir: .pytest_cache
rootdir: C:\Users\User\My_Project\HW_Python
plugins: anyio-4.3.0
collecting ... collected 8 items

..\tests\test_scraper.py::test_book_data_dict PASSED                     [ 12%]
..\tests\test_scraper.py::test_book_data_has_all_keys PASSED             [ 25%]
..\tests\test_scraper.py::test_book_data_name_correct PASSED             [ 37%]
..\tests\test_scraper.py::test_book_data_price_correct PASSED            [ 50%]
..\tests\test_scraper.py::test_get_book_data_rating_correct PASSED       [ 62%]
..\tests\test_scraper.py::test_book_data_extra_features_correct PASSED   [ 75%]
..\tests\test_scraper.py::test_scrape_books_returns_list PASSED          [ 87%]
..\tests\test_scraper.py::test_scrape_books_count_correct PASSED         [100%]

======================== 8 p

## Задание 5. Оформление проекта на GitHub и работа с Git (35 баллов)

В этом задании нужно воспользоваться системой контроля версий Git и платформой GitHub для хранения и управления своим проектом. **Ссылку на свой репозиторий пришлите в форме для сдачи ответа.**

### Пошаговая инструкция и задания

**1. Установите Git на свой компьютер.**

* Для Windows: [скачайте установщик](https://git-scm.com/downloads) и выполните установку.
* Для macOS:

  ```
  brew install git
  ```
* Для Linux:

  ```
  sudo apt update
  sudo apt install git
  ```

**2. Настройте имя пользователя и email.**

Это нужно для подписи ваших коммитов, сделайте в терминале через `git config ...`.

**3. Создайте аккаунт на GitHub**, если у вас его еще нет:
[https://github.com](https://github.com)

**4. Создайте новый репозиторий на GitHub:**

* Найдите кнопку **New repository**.
* Укажите название, краткое описание, выберите тип **Public** (чтобы мы могли проверить ДЗ).
* Не ставьте галочку Initialize this repository with a README.

**5. Создайте локальную папку с проектом.** Можно в терминале, можно через UI, это не имеет значения.

**6. Инициализируйте Git в этой папке.** Здесь уже придется воспользоваться некоторой командой в терминале.

**7. Привяжите локальный репозиторий к удаленному на GitHub.**

**8. Создайте ветку разработки.** По умолчанию вы будете находиться в ветке `main`, создайте и переключитесь на ветку `hw-books-parser`.

**9. Добавьте в проект следующие файлы и папки:**

* `scraper.py` — ваш основной скрипт для сбора данных.
* `README.md` — файл с кратким описанием проекта:

  * цель;
  * инструкции по запуску;
  * список используемых библиотек.
* `requirements.txt` — файл со списком зависимостей, необходимых для проекта (не присылайте все из глобального окружения, создайте изолированную виртуальную среду, добавьте в нее все нужное для проекта и получите список библиотек через `pip freeze`).
* `artifacts/` — папка с результатами парсинга (`books_data.txt` — полностью или его часть, если весь не поместится на GitHub).
* `notebooks/` — папка с заполненным ноутбуком `HW_03_python_ds_2025.ipynb` и запущенными ячейками с выводами на экран.
* `tests/` — папка с тестами на `pytest`, оформите их в формате скрипта(-ов) с расширением `.py`.
* `.gitignore` — стандартный файл, который позволит исключить временные файлы при добавлении в отслеживаемые (например, `__pycache__/`, `.DS_Store`, `*.pyc`, `venv/` и др.).


**10. Сделайте коммит.**

**11. Отправьте свою ветку на GitHub.**

**12. Создайте Pull Request:**

* Перейдите в репозиторий на GitHub.
* Нажмите кнопку **Compare & pull request**.
* Укажите, что было добавлено, и нажмите **Create pull request**.

**13. Выполните слияние Pull Request:**

* Убедитесь, что нет конфликтов.
* Нажмите **Merge pull request**, затем **Confirm merge**.

**14. Скачайте изменения из основной ветки локально.**



### Требования к итоговому репозиторию

* Файл `scraper.py` с рабочим кодом парсера.
* `README.md` с описанием проекта и инструкцией по запуску.
* Папка `artifacts/` с результатом сбора данных (`.txt` файл).
* Папка `tests/` с тестами на `pytest`.
* Папка `notebooks/` с заполненным ноутбуком `HW_03_python_ds_2025.ipynb`.
* Pull Request с комментарием из ветки `hw-books-parser` в ветку `main`.
* Примерная структура:

  ```
  books_scraper/
  ├── artifacts/
  │   └── books_data.txt
  ├── notebooks/
  │   └── HW_03_python_ds_2025.ipynb
  ├── scraper.py
  ├── README.md
  ├── tests/
  │   └── test_scraper.py
  ├── .gitignore
  └── requirements.txt
  ```